# Imports 

In [1]:
"""

pip install scipy
pip install h5py
pip install matplotlib
pip install opencv-python
pip install seaborn
pip install matplotlib
pip install ipywidgets
pip install matplotlib install ipywidgets
pip install opencv-pythonjupyter nbextension enable --py widgetsnbextension
pip install ipython
pip install WordCloud
pip install geopy
pip install scipy
pip install plotly
pip install nbformat --upgrade
pip install folium
pip install geopandas
pip install scikit-learn   
pip install cairosvg
pip install lxm



pip install selenium
pip install pandas
pip install python-dotenv
pip install openai
pip install beautifulsoup4
pip install numpy
pip install tqdm
pip install seaborn
pip install matplotlib
pip install scipy
pip install geopy
pip install plotly



"""

'\n\npip install scipy\npip install h5py\npip install matplotlib\npip install opencv-python\npip install seaborn\npip install matplotlib\npip install ipywidgets\npip install matplotlib install ipywidgets\npip install opencv-pythonjupyter nbextension enable --py widgetsnbextension\npip install ipython\npip install WordCloud\npip install geopy\npip install scipy\npip install plotly\npip install nbformat --upgrade\npip install folium\npip install geopandas\npip install scikit-learn   \npip install cairosvg\npip install lxm\n\n\n\npip install selenium\npip install pandas\npip install python-dotenv\npip install openai\npip install beautifulsoup4\npip install numpy\npip install tqdm\npip install seaborn\npip install matplotlib\npip install scipy\npip install geopy\npip install plotly\n\n\n\n'

In [2]:
gpt_instructions = """
for the below text using using your LLM ability (not python code) extract data from the text. get the following and save all as strings, if they don't exist for a specific job entry, use an empty string (i.e., ""). 
there will be exactly 20 of each. 

job_title 
company_name 
location

format the output as a python single dictionary for all the jobs. for example all the job titles will use key "job_title" and be a list of the jobs

I need to have have a standard output and use it directly for my analysis. process every single job 

Construct the dictionary: Organize all the extracted information into a dictionary format where the key represents the type of information and the value is a list of all the extracted information of that type in the order of appearance.

Formatting the instructions: Always make sure to start the code block with a comment '##BEGIN##', fill in the data accordingly, and end it with '##END##'. All columns of data should align with each other.

    For example:


    ##BEGIN##
job_data = {
    "job_title": [...],
    "company_name": [...],
    "location": [...]
}
    ##END##

    Please make sure to follow these instructions to achieve consistent results.

    here is the text:



    
"""

In [3]:

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd
from dotenv import load_dotenv
import os
import openai

from bs4 import BeautifulSoup
import re
import time
import pickle
import numpy as np 
import pandas as pd

from scrapifurs import utils
from scrapifurs.GPTinstructions import GPTinstructions

from tqdm import tqdm

import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.cluster.hierarchy import linkage, leaves_list

from geopy.geocoders import Nominatim
from geopy.distance import geodesic


import plotly.express as px

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime





In [4]:
def get_lat_lon(locations):
    geolocator = Nominatim(user_agent="example.lastname@gmail.com")
    lat_lon_data = {}
    for location in tqdm(locations):
        location_data = geolocator.geocode(location)
        if location_data is not None:
            lat_lon_data[location] = (location_data.latitude, location_data.longitude)
        else:
            lat_lon_data[location] = (None, None)
    return lat_lon_data

def calculate_distances(lat_lon_data, target_loc_str):
    target_location = get_lat_lon([target_loc_str])[target_loc_str]
    if target_location[0] is None:
        raise ValueError("No latitude and longitude found for the provided location.")
    distances = {}
    for location, loc_lat_lon in lat_lon_data.items():
        if loc_lat_lon[0] is not None:
            distance = geodesic(target_location, loc_lat_lon).miles
            distances[location] = distance
        else:
            distances[location] = None
    return distances





def jaccard_similarity(str1, str2):
    set1 = set(str1.split())
    set2 = set(str2.split())
    union_length = len(set1.union(set2))
    if union_length == 0:
        return np.nan
    return len(set1.intersection(set2)) / union_length


class GPT_StringDataCleaner:
    def __init__(self, original_df, key_name, instructions, overwrite_cleaned_key = True, model="gpt-4"):
        self.original_df = original_df
        self.key_name = key_name
        self.instructions = instructions
        self.model = model
        self.clean_key = key_name + '_CLEANED_BY_GPT'
        self.overwrite_cleaned_key = overwrite_cleaned_key

    def send_to_gpt(self):
        self.unique_names = sorted(self.original_df[self.key_name].unique())

        messages = [{"role": "system", "content": self.instructions}]
        messages.append({"role": "user", "content": str(self.unique_names)})
        chat = openai.ChatCompletion.create(model=self.model, messages=messages)
        self.reply = chat.choices[0].message.content
    def test_lengths(self):
        print(f" len input is {len(self.unique_names)}, but len output is {len(cleaned_data)}")
    def process_data(self):
        cleaned_data =  eval(self.reply)
        
        if len(self.unique_names) != len(cleaned_data):
            print(f" len input is {len(self.unique_names)}, but len output is {len(cleaned_data)}")
            raise ValueError("Input and output lists must have the same length")

        temp_df = pd.DataFrame({self.key_name: self.unique_names, self.clean_key: cleaned_data})
        # delete cleaned key data if we want to overwrite
        if self.clean_key in self.original_df.columns and self.overwrite_cleaned_key:
            self.original_df = self.original_df.drop(self.clean_key, axis=1)
        if self.clean_key not in self.original_df.columns:
            merged_df = pd.merge(self.original_df, temp_df[[self.clean_key, self.key_name]],
                                 on=self.key_name, how='left')
            self.original_df[self.clean_key] = merged_df[self.clean_key]
            merged_df = None

        return self.original_df

In [5]:

def update_master_df(job_data_all, job_data):
    # Reverse the order of job_data to check oldest entries first
    job_data_reversed = job_data.iloc[::-1]
    
    # List to hold new entries
    new_entries = []
    
    for i, row in job_data_reversed.iterrows():
        # Check if row exists in job_data_all
        is_exist = job_data_all[(job_data_all['job_title'] == row['job_title']) & 
                                (job_data_all['company_name'] == row['company_name']) &
                                (job_data_all['location'] == row['location'])].shape[0]
        
        # If row does not exist in job_data_all, add it to new_entries
        if is_exist == 0:
            print('NEWWWWWW')
            print(row)
            new_entries.append(row)
    
    # Convert new_entries to DataFrame and concatenate it with job_data_all
    if new_entries:
        new_entries_df = pd.DataFrame(new_entries)
        job_data_all = pd.concat([new_entries_df, job_data_all], ignore_index=True)
    
    return job_data_all


def get_job_blocks(gpt_text, job_data):
    job_blocks = []
    combined_titles = "|".join([re.escape(title) for title in job_data['job_title']])
    combined_companies = "|".join([re.escape(company) for company in job_data['company_name']])
    pattern = f"({combined_titles}).*?({combined_companies}).*?(?=({combined_titles}).*?({combined_companies})|$)"
    
    for match in re.finditer(pattern, gpt_text, re.DOTALL):
        job_blocks.append(match.group(0))

    return job_blocks

In [48]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def open_browser(info_dict):
    #init chrome 
    chrome_options = Options()
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--no-sandbox")
    driver = webdriver.Chrome(options=chrome_options)
    
    driver.get(info_dict['init_url'])
    time.sleep(1)
    driver.get(info_dict['init_url'])
    time.sleep(2)
    
    
    # Load cookies if they exist
    try:
        cookies = pickle.load(open(info_dict['full_cookies_save_path'], "rb"))
        for cookie in cookies:
            driver.add_cookie(cookie)
        driver.refresh()
        assert(not not cookies)# if empty try a different method
    except:
        print("No cookies found. Manual login required.")
        # If not logged in
        input('Please login and press Enter to continue...')
        pickle.dump(driver.get_cookies(), open(info_dict['full_cookies_save_path'], "wb")) # save cookies after login
    time.sleep(4)
    return driver
        
    
    time.sleep(4)
def go_to_next_page(driver):
    try:
        # Step 1: Find the pagination element
        pagination = driver.find_element(By.CSS_SELECTOR, 'ul.artdeco-pagination__pages')
    except NoSuchElementException:
        print("Pagination not found")
        return

    try:
        # Step 2: Find the current active page
        current_page_elem = pagination.find_element(By.CSS_SELECTOR, 'li.active')
        current_page = int(current_page_elem.text)
    except NoSuchElementException:
        print("Current page not found")
        return

    # Step 3: Find the next page
    next_page = current_page + 1
    next_page_selector = f'li[data-test-pagination-page-btn="{next_page}"]'

    try:
        next_page_elem = pagination.find_element(By.CSS_SELECTOR, next_page_selector)
    except NoSuchElementException:
        print("Next page not found")
        return

    # Step 4: Click the next page
    next_page_elem.click()


from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from datetime import datetime

def update_dataframe(df_main, df_new, keys=None):
    if keys is None:
        keys = ['job_ids', 'job_title', 'company_name', 'Location', 'pay']

    # Check if types are the same for each key
    mismatched_keys = []
    for key in keys:
        type_main = df_main[key].dtype
        type_new = df_new[key].dtype
        if type_main != type_new:
            mismatched_keys.append((key, type_main, type_new))

    if mismatched_keys:
        print("Type mismatch for these keys:")
        for key, type_main, type_new in mismatched_keys:
            print(f"Key: {key}, Type in df_main: {type_main}, Type in df_new: {type_new}")
        raise TypeError("Type mismatch detected.")
        
    for _, new_row in df_new.iterrows():
        mask = df_main[keys].eq(new_row[keys]).all(axis=1)
        if not any(mask):
            df_main = pd.concat([df_main, pd.DataFrame([new_row])], ignore_index=True)
            os.system('say "youve got jobs"')  # macOS
            print('\nNNNNNEEWWWWWWW')
            print(new_row['job_link'])
            print(new_row)

    return df_main


# def update_dataframe(df_main, df_new, keys=None):
#     if keys is None:
#         # keys = df_main.columns.tolist()
#         keys = ['job_ids', 'job_title', 'company_name', 'Location', 'pay']
        
        
#     for i, new_row in df_new.iterrows():
#         if not any(df_main[keys].eq(new_row[keys]).all(axis=1)):
            
#             df_main = pd.concat([df_main, pd.DataFrame([new_row])], ignore_index=True)
#             print('\nNNNNNEEWWWWWWW')
#             print('\n')
#             print(new_row['job_link'])
#             print(new_row)
#             asdfasdfasdf
#     return df_main

from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

def get_job_details(driver):
    job_dict = {
        "job_ids": [],
        "is_promoted": [],
        "job_title": [],
        "company_name": [],
        "Location": [],
        "pay": [],
        "job_link": [],
        "time_added": [],
    }
    
    jobs = driver.find_elements(By.CSS_SELECTOR, '[data-occludable-job-id]')
    
    for job in jobs:
        try:
            job_id = job.get_attribute("data-occludable-job-id")
        except NoSuchElementException:
            job_id = -1
        
        try:
            promoted_elements = job.find_elements(By.CSS_SELECTOR, '.job-card-container__footer-item')
            is_promoted = "Promoted" in [e.text for e in promoted_elements]
        except NoSuchElementException:
            is_promoted = False

        try:
            job_title = job.find_element(By.CSS_SELECTOR, '.job-card-list__title').text
        except NoSuchElementException:
            job_title = ""

        try:
            company_name = job.find_element(By.CSS_SELECTOR, '.job-card-container__primary-description').text
        except NoSuchElementException:
            company_name = ""

        try:
            Location = job.find_element(By.CSS_SELECTOR, '.job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            Location = ""

        try:
            pay = job.find_element(By.CSS_SELECTOR, '.mt1 .job-card-container__metadata-wrapper li').text
        except NoSuchElementException:
            pay = ""
            
        try:
            job_link = job.find_element(By.CSS_SELECTOR, "a.job-card-container__link").get_attribute("href")
        except NoSuchElementException:
            job_link = ""
            
        

        if job_title != "" and company_name != "":
            job_dict["job_ids"].append(job_id)
            job_dict["is_promoted"].append(is_promoted)
            job_dict["job_title"].append(job_title)
            job_dict["company_name"].append(company_name)
            job_dict["Location"].append(Location)
            job_dict["pay"].append(pay)
            job_dict["job_link"].append(job_link)
            
            current_time = datetime.now()
            formatted_time = current_time.strftime('%Y%m%d%H%M%S')
            formatted_time_as_int = int(formatted_time)
            job_dict["time_added"].append(formatted_time_as_int)
            
        
    return job_dict

class data_file():

    def __init__(self, fn):
        self.fn = fn
        self.ft = fn.split('.')[-1].lower()
        self.exists = self.check_if_exists()
        self.df_main = None
        self.load_it()
    
    def check_if_exists(self):
        return os.path.exists(self.fn)
    
    def load_it(self):
    
        try:
            self.df_main = pd.read_excel(self.fn) 
        except:
            try:
                self.df_main = pd.read_csv(self.fn)
            except:
                print(f"File type for {self.fn} is unsupported")
        
    def save_it(self):
        # larger job IDs are always more recent. 
        # self.df_main = self.df_main.sort_values(by='job_ids', ascending=False) 
        self.df_main = self.df_main.sort_values(by='job_ids', ascending=False).reset_index(drop=True)
        

        try:
            self.df_main.to_excel(self.fn, index=False)
        except:
            try: 
                self.df_main.to_csv(self.fn, index=False)  
            except:
                print(f"Error saving {self.fn}")

### INIT API key,  intrucitons for GPT which can be saved as text files in the correct data/instrucitons dir of the package

In [7]:

# # setup API key for chatGPT 
# load_dotenv()  # take environment variables from .env.
# os.environ["OPENAI_API_KEY"]  = os.getenv("OPENAI_API_KEY")
# openai.api_key = os.environ["OPENAI_API_KEY"]




#### navigate to the first page we want to scrape

In [45]:
import time
import pandas as pd

# Assuming DataClass is defined elsewhere and has methods df_main, save_it()

def scrape_job_data(info_dict, data_class, n_pages_to_scrape=25, wait_sec_each_page=5, update_every_n_secs=360):
    """
    Scrapes job data for a specified number of pages and intervals.

    Args:
    - info_dict: A dictionary containing start_url and other necessary information.
    - data_class: An instance of a DataClass containing df_main and a save_it method.
    - n_pages_to_scrape: Number of pages to scrape.
    - wait_sec_each_page: Time to wait on each page before scraping.
    - update_every_n_secs: How often to update the data in seconds.
    """

    driver = open_browser(info_dict)
    driver.get(info_dict['start_url'])

    try:
        while True:
            for n_page in range(n_pages_to_scrape):
                if n_page == 0:
                    # Go back to the first page
                    driver.get(info_dict['start_url'])
                else:
                    go_to_next_page(driver)
                time.sleep(wait_sec_each_page)
                # Update data
                job_data = get_job_details(driver)
                job_data = pd.DataFrame(job_data)
                job_data['job_ids'] = job_data['job_ids'].astype('int')
                data_class.df_main = update_dataframe(data_class.df_main, job_data, ['job_ids'])
            
                data_class.save_it()
            
            print('_____________________________________________\n\n____________________')
            time.sleep(update_every_n_secs)
    finally:
        driver.quit()



# Info_dict to set up all used variables

In [8]:
# setup basic variable as dict 
info_dict = {'init_url':'https://www.linkedin.com/',
             'save_password_dir':'/Users/phil/Dropbox/GITHUB/DATA/scrapifurs/saved_cookies/',
             'start_url':'https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=fRq'}
info_dict['full_cookies_save_path'] = info_dict['save_password_dir']+os.sep+"linkedin_cookies.pkl"


In [ ]:
# for the recomended jobs file


x = 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3758777094'
x = 'https://www.linkedin.com/jobs/collections/recommended/?currentJobId=3759374185&origin=JOB_ALERT_IN_APP_NOTIFICATION&originToLandingJobPostings=3752871491%2C3759374185'
info_dict['start_url'] = x
fn = '/Users/phil/Library/CloudStorage/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/recommended_jobs.xlsx'
data_class = data_file(fn)     
scrape_job_data(info_dict, data_class, n_pages_to_scrape=25, wait_sec_each_page=5, update_every_n_secs=60*15)



In [47]:
# # for the job search 

# x = 'https://www.linkedin.com/jobs/search/?currentJobId=3742729997&distance=25&f_PP=102277331%2C106233382%2C102448103%2C103918656%2C102250832%2C103575230%2C100075706&f_T=25206%2C340%2C25190%2C25887%2C30209%2C288%2C2463%2C25584&f_WT=1&geoId=103644278&keywords=data%20scientist&origin=JOBS_HOME_SEARCH_CARDS'
# info_dict['start_url'] = x
# fn = '/Users/phil/Library/CloudStorage/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/TEST_V1.xlsx'
# data_class = data_file(fn)     
# scrape_job_data(info_dict, data_class, n_pages_to_scrape=9, wait_sec_each_page=5, update_every_n_secs=60*15)





In [42]:
data_class.df_main

data_class.df_main.sort_values(by='job_ids', ascending=False).reset_index(drop=True)



,job_ids,is_promoted,job_title,company_name,Location,pay,job_link,time_added
0,3761112021,False,Data Scientist,Actalent,"Mountain View, CA (On-site)",,https://www.linkedin.com/jobs/view/3761112021/...,20231108101411
1,3761064412,False,"Data Scientist, Junior with Security Clearance",ClearanceJobs,"San Diego, CA (Remote)","$58K/yr - $133K/yr · 401(k), +1 benefit",https://www.linkedin.com/jobs/view/3761064412/...,20231108101337
2,3760987820,False,Data Analyst (Churn),Harnham,"Los Angeles, CA (Remote)",$40/hr - $50/hr,https://www.linkedin.com/jobs/view/3760987820/...,20231108101302
3,3760985956,False,Senior Machine Learning Software Engineer,Harnham,"San Diego, CA",$160K/yr - $200K/yr,https://www.linkedin.com/jobs/view/3760985956/...,20231108101302
4,3760978153,False,Machine Learning Engineer Graduate (Monetizati...,TikTok,"San Jose, CA","$145K/yr - $250K/yr · Medical, 401(k), +1 benefit",https://www.linkedin.com/jobs/view/3760978153/...,20231108101302
...,...,...,...,...,...,...,...,...
1261,3219151495,True,Data Scientist,ZipRecruiter,"Los Angeles, CA (Remote)",$105K/yr - $145K/yr ¬∑ 401(k) benefit,https://www.linkedin.com/jobs/view/3219151495/...,20231006170727
1262,3219151495,True,Data Scientist,ZipRecruiter,"Los Angeles, CA (Remote)",401(k) benefit,https://www.linkedin.com/jobs/view/3219151495/...,20231009095401
1263,3137433250,True,"Data Scientist- TikTok Ads, Ad Inventory & Use...",TikTok,"Mountain View, CA","$153K/yr - $296K/yr ¬∑ Medical, 401(k), +1 ben...",https://www.linkedin.com/jobs/view/3137433250/...,20231006181158
1264,3091135285,True,"Staff Machine Learning Engineer, Perception",Cruise,"San Francisco, CA","$193K/yr - $285K/yr ¬∑ 401(k), +1 benefit",https://www.linkedin.com/jobs/view/3091135285/...,20231006163510


In [ ]:
import re
import pandas as pd

# Sample data
data = [
    "$184K/yr - $338K/yr ¬∑ Medical, 401(k), +1 benefit",
    # More data...
]

# Function to extract salary information
def extract_salary_info(text):
    # Find all patterns of money amounts followed by either "hr" or "yr"
    patterns = re.findall(r'\$(\S+)(?= |hr|yr)', text)
    time_basis = re.findall(r'per (hr|yr)', text.lower())

    # Convert to float and scale numbers ending in 'K'
    amounts = [float(amount.rstrip('K')) * 1000 if 'K' in amount else float(amount) for amount in patterns]
    # Determine min and max salaries
    min_salary = min(amounts) if amounts else None
    max_salary = max(amounts) if amounts else None
    # Determine time basis ('hr' or 'yr'), default to None if not found
    per_time = time_basis[0] if time_basis else None

    return min_salary, max_salary, per_time

# Apply the function to each item in the data list and create a DataFrame
extracted_data = [extract_salary_info(item) for item in data]
df = pd.DataFrame(extracted_data, columns=['Min_Salary', 'Max_Salary', 'Per'])

print(df)


In [35]:
# job_data = get_job_details(driver)
# df_main = pd.DataFrame(job_data)
# df_main

driver = open_browser(info_dict)
driver.get(info_dict['start_url'])

n_pages_to_scrape = 25
wait_sec_each_page = 10-5
update_every_n_secs = 60*6


while True:
    for n_page in range(n_pages_to_scrape):
        if n_page ==0:
            # go back to the first page
            driver.get(info_dict['start_url'])
        else:
            go_to_next_page(driver)
        time.sleep(wait_sec_each_page)
        # update data
        job_data = get_job_details(driver)
        job_data = pd.DataFrame(job_data)
        job_data['job_ids'] = job_data['job_ids'].astype('int')
        data_class.df_main = update_dataframe(data_class.df_main, job_data, ['job_ids'])
    
    # save CSV 
    # df_main.to_csv('/Users/phil/Dropbox/GITHUB/scrapifurs/scrapifurs/data/tempdata/DS_mega_data_V1.csv', index=False)
    data_class.save_it()
    
    print('_____________________________________________\n\n____________________')
    time.sleep(update_every_n_secs)



NNNNNEEWWWWWWW
https://www.linkedin.com/jobs/view/3758773807/?eBP=JYMBII_JOBS_HOME_ORGANIC&refId=RUCoRui4ajkjVdI7Fpu2lQ%3D%3D&trackingId=pzN%2FSASXpitD%2F6HaiQ5tZA%3D%3D&trk=flagship3_jobs_discovery_jymbii
job_ids                                                3758773807
is_promoted                                                 False
job_title       Artificial Intelligence Engineer, Trilogy (Rem...
company_name                                            Crossover
Location                         New York, United States (Remote)
pay                                                      $200K/yr
job_link        https://www.linkedin.com/jobs/view/3758773807/...
time_added                                         20231108064242
Name: 0, dtype: object

NNNNNEEWWWWWWW
https://www.linkedin.com/jobs/view/3756110225/?eBP=CwEAAAGLr2KdnRFfiWiSKukM5FhkU4gTQPLUEyPDqRiJPmtavJkJ9bGdVqNtbVrOXvoPEzJSqwvFQjTUt4SF7pcbK3IfGtoefm1cwRUseoNEXbLm3orRsyLZxnhoboX63s60U4yuE4fMsUVUv_cg6q3sJGxFTkOjoHxWOOTARQznzGi

KeyboardInterrupt: 

# Scraping step

### create rules to trim text based on key workds and search characteristics
this willl help reduce the amount of data we pass to GPT API and reduce costs 

In [9]:
# text_finder = utils.StringSectionExtractor()
# text_finder.add_start_rule('\d+ results', True)
# text_finder.add_start_rule('All filters', False)
# text_finder.add_start_rule('Jump to active job details', False)


# text_finder.add_end_rule('Page \d+ of \d+', True)
# text_finder.add_end_rule("Are you finding", False)

### INIT: chrome browser, login, save cookies (future login)